# Лабораторна робота 3. Машинне навчання — Лінійна регресія
**Підготовлено:** Кучеренко Іван  
**Варіант:** 7

Файл містить два завдання:
- Завдання 1: Diabetes Dataset — попередній аналіз, побудова моделей LinearRegression та RandomForestRegressor.
- Завдання 2: Housing dataset (файл `house_price_regression_dataset.csv`) — попередній аналіз, побудова моделей LinearRegression, RandomForestRegressor та Ridge з підбором гіперпараметрів.

У кожному завданні код містить кроки: зчитування/огляд даних, обробка пропусків, видалення дублікатів, масштабування, розбиття на трен/тест, навчання моделей, оцінка (R², MSE), графіки та висновок.


## Завдання 1 — Diabetes Dataset
**Кроки:** 1) Зчитати датасет; 2) Аналіз даних; 3) Обробка; 4) Кореляція; 5) Масштабування; 6) Розбиття; 7) Навчання LinearRegression та RandomForest; 8) Оцінка; 9) Графіки; 10) Висновок.

In [ ]:

# Завдання 1 — Diabetes Dataset
# 1. Імпорт бібліотек та завантаження датасету
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Завантажуємо датасет
diab = load_diabetes(as_frame=True)
X = diab.data
y = diab.target
df = pd.concat([X, y.rename("target")], axis=1)

# 1. Перші 5 рядків
print("Перші 5 рядків датасету Diabetes:")
display(df.head())

# 2. Перевірка на пропуски
print("\nКількість пропусків по стовпцях:")
print(df.isna().sum())

# Запис: у цьому датасеті пропусків немає, але на випадок - заповнимо середнім (загальний шаблон)
df = df.fillna(df.mean(numeric_only=True))

# 3. Перевірка дублікатів
dups = df.duplicated().sum()
print(f"\nКількість дублікатів: {dups}")
if dups > 0:
    df = df.drop_duplicates()
    print("Дублікат(и) видалено.")

# 4. Перевірка бінарних ознак (стать) — у diabetes стандартних label'ів немає, але перевіримо уявлення
print("\nОригінальні колонки:")
print(list(df.columns))

# (У цьому датасеті 'sex' є нормалізованою числовою ознакою; для безпечності перевіримо унікальні значення)
if 'sex' in df.columns:
    print("\nУнікальні значення 'sex':", df['sex'].unique())

# 5. Типи даних
print("\nТипи даних:")
print(df.dtypes)

# 6. Кореляція з цільовою змінною (прогрес діабету)
corrs = df.corr()['target'].drop('target').abs().sort_values(ascending=False)
print("\nКореляція ознак з target (у порядку спадання):")
display(corrs)

# Побудуємо теплову карту кореляцій (всіх змінних)
plt.figure(figsize=(10,8))
corr_matrix = df.corr()
plt.imshow(corr_matrix, cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)
plt.title('Теплова карта кореляцій (Diabetes)')
plt.tight_layout()
plt.show()

# 7. Масштабування ознак
features = df.drop(columns=['target']).columns.tolist()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])
X_scaled = pd.DataFrame(X_scaled, columns=features)

# 8. Розбиття на тренувальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['target'], test_size=0.2, random_state=42)

# 9. Вибір найбільш корельованих ознак (візьмемо top-5)
top_k = 5
top_features = corrs.head(top_k).index.tolist()
print(f"\nВибрані {top_k} найбільш корельованих ознак:", top_features)

X_train_sel = X_train[top_features]
X_test_sel = X_test[top_features]

# Навчання моделей
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42, n_estimators=100)

lr.fit(X_train_sel, y_train)
rf.fit(X_train_sel, y_train)

# 10. Оцінка
y_pred_lr = lr.predict(X_test_sel)
y_pred_rf = rf.predict(X_test_sel)

r2_lr = r2_score(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)

r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)

print("\nLinear Regression: R2 = {:.4f}, MSE = {:.4f}".format(r2_lr, mse_lr))
print("Random Forest:     R2 = {:.4f}, MSE = {:.4f}".format(r2_rf, mse_rf))

# 11. Графіки: справжні vs прогнозовані
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(y_test, y_pred_lr, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('True target')
plt.ylabel('Predicted (LR)')
plt.title('True vs Predicted — LinearRegression')

plt.subplot(1,2,2)
plt.scatter(y_test, y_pred_rf, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('True target')
plt.ylabel('Predicted (RF)')
plt.title('True vs Predicted — RandomForest')
plt.tight_layout()
plt.show()

# Графік залишків для кращого розуміння
res_lr = y_test - y_pred_lr
res_rf = y_test - y_pred_rf
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.scatter(y_pred_lr, res_lr, alpha=0.7)
plt.axhline(0, color='k', linestyle='--')
plt.xlabel('Predicted (LR)')
plt.ylabel('Residuals')
plt.title('Residuals — LinearRegression')

plt.subplot(1,2,2)
plt.scatter(y_pred_rf, res_rf, alpha=0.7)
plt.axhline(0, color='k', linestyle='--')
plt.xlabel('Predicted (RF)')
plt.ylabel('Residuals')
plt.title('Residuals — RandomForest')
plt.tight_layout()
plt.show()

# 12. Вивести справжні і прогнозовані значення (перші 10)
results = pd.DataFrame({
    'True': y_test.values,
    'Pred_LR': y_pred_lr,
    'Pred_RF': y_pred_rf
}).reset_index(drop=True)
print("\nПерші 10 рядків: справжні та прогнозовані значення")
display(results.head(10))

# 13. Висновок (коментар)
print("\nВИСНОВОК:\n- На датасеті Diabetes LinearRegression та RandomForest показали порівнянні результати.\n- RandomForest може давати кращу нелінійну апроксимацію, що видно з трохи вищого R2 або нижчого MSE (залежить від випадку).\n- Вибір найбільш корельованих ознак (top-k) допоміг скоротити розмір задачі та зберегти інформативність.")


## Завдання 2 — Housing dataset (файл `house_price_regression_dataset.csv`)
Кроки: 1) Зчитати CSV; 2) Аналіз; 3) Обробка пропусків, дублікатів; 4) Масштабування; 5) Розбиття; 6) Навчання Linear, RandomForest, Ridge з GridSearch; 7) Оцінка; 8) Графіки; 9) Висновки.

In [ ]:

# Завдання 2 — Housing dataset з файлу 'house_price_regression_dataset.csv'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Зчитування CSV (файл має бути в /mnt/data)
csv_path = "/mnt/data/house_price_regression_dataset.csv"
print(f"Читаємо файл: {csv_path}")
housing = pd.read_csv(csv_path)

# 1. Перші 5 рядків
print("\nПерші 5 рядків датасету Housing:")
display(housing.head())

# 2. Попередній аналіз: типи, розмір, пропуски
print("\nРозмір датасету:", housing.shape)
print("\nТипи даних:")
print(housing.dtypes)
print("\nКількість пропусків по стовпцях:")
print(housing.isna().sum())

# 3. Обробка пропусків: заповнюємо середнім для числових колонок
numeric_cols = housing.select_dtypes(include=[np.number]).columns.tolist()
housing[numeric_cols] = housing[numeric_cols].fillna(housing[numeric_cols].mean())

# 4. Перевірка дублікатів
dups = housing.duplicated().sum()
print(f"\nКількість дублікатів: {dups}")
if dups > 0:
    housing = housing.drop_duplicates()
    print("Дублікат(и) видалено.")

# 5. Оберіть цільову змінну. Якщо в датасеті є 'Price' або 'SalePrice' - використаємо її. 
possible_targets = [c for c in housing.columns if c.lower() in ('price','saleprice','median_house_value','medianvalue','target','y')]
print("\nМожливі кандидатури на цільову змінну:", possible_targets)

if len(possible_targets) > 0:
    target_col = possible_targets[0]
else:
    # якщо немає очевидного, візьмемо останню числову колонку як target (попередження)
    numeric_cols = housing.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) == 0:
        raise ValueError("У датасеті немає числових колонок для регресії.")
    target_col = numeric_cols[-1]
print("Використовуємо як цільову змінну:", target_col)

# Підготовка X та y
X = housing.drop(columns=[target_col])
y = housing[target_col]

# Перетворимо категоріальні змінні (якщо є) за допомогою one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Впевненість: видаляємо колонки з NaN (якщо залишилися)
X = X.fillna(0)

# 6. Масштабування
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# 7. Розбиття
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 8. Навчання моделей: LinearRegression, RandomForest, Ridge (з GridSearch)
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42, n_estimators=200)

# Підбір гіперпараметрів для Ridge та RandomForest (GridSearch для Ridge; для RF аж ніяк не повний пошук, лише приклад)
ridge = Ridge()
ridge_params = {'alpha': [0.1, 1.0, 10.0, 50.0]}

rf_params = {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}

# GridSearch для Ridge
gs_ridge = GridSearchCV(ridge, ridge_params, cv=4, scoring='r2', n_jobs=-1)
gs_ridge.fit(X_train, y_train)
best_ridge = gs_ridge.best_estimator_
print("\nНайкращий Ridge:", gs_ridge.best_params_)

# GridSearch короткий для RF (щоб не затягувати)
gs_rf = GridSearchCV(rf, rf_params, cv=3, scoring='r2', n_jobs=-1)
gs_rf.fit(X_train, y_train)
best_rf = gs_rf.best_estimator_
print("Найкращий RF params:", gs_rf.best_params_)

# Навчання LinearRegression
lr.fit(X_train, y_train)

# 9. Оцінка моделей
models = {
    'LinearRegression': lr,
    'RandomForest': best_rf,
    'Ridge': best_ridge
}

scores = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    scores[name] = (r2, mse)
    print(f"\n{name}: R2 = {r2:.4f}, MSE = {mse:.4f}")

# 10. Графіки: True vs Predicted для кожної моделі
plt.figure(figsize=(15,5))
for i, (name, model) in enumerate(models.items(), 1):
    y_pred = model.predict(X_test)
    plt.subplot(1,3,i)
    plt.scatter(y_test, y_pred, alpha=0.5)
    mn, mx = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
    plt.plot([mn,mx],[mn,mx],'k--')
    plt.xlabel('True')
    plt.ylabel('Predicted')
    plt.title(f'{name}\nR2={scores[name][0]:.3f}, MSE={scores[name][1]:.3f}')
plt.tight_layout()
plt.show()

# 11. Вивести справжні і прогнозовані значення (перші 10)
for name, model in models.items():
    preds = model.predict(X_test)[:10]
    print(f"\n{name} — перші 10 прогнозів:")
    display(pd.DataFrame({'True': y_test.values[:10], 'Pred': preds}))

# 12. Висновки
print("\nВИСНОВОК:\n- Було виконано попередній аналіз та обробку датасету житла.\n- Застосовано три моделі: LinearRegression, RandomForest (з GridSearch), Ridge (з GridSearch).\n- Вибір найкращої моделі залежить від метрики R2 та MSE; RandomForest зазвичай дає кращі результати на складних неформальних зв'язках, але ризикує перенавчитися.\n- Перевіряйте важливість ознак та крос-валідацію для стабільної оцінки.")


## Загальні висновки по лабораторній роботі 3

- Ми виконали попередній аналіз даних на двох наборах: Diabetes (sklearn) та Housing (з наданого CSV).
- Продемонстровано обробку пропусків, видалення дублікатів, масштабування ознак, розбиття на трен/тест, побудову моделей та їх оцінку.
- RandomForest зазвичай показує кращу якість на складних залежностях, тоді як лінійні моделі (LinearRegression, Ridge) легші для інтерпретації.
- Для реального проєкту рекомендовано проводити більш ретельний відбір ознак, крос-валідацію та налаштування гіперпараметрів.

### Примітки
- Якщо файл `house_price_regression_dataset.csv` має іншу назву або іншу структуру, потрібно вказати правильну цільову змінну (стовпець з ціною).
- В ноутбуці використано GridSearch з невеликими сітками для прикладу — за потреби розширюйте параметри.
